<a href="https://colab.research.google.com/github/shinb-bong/TIL/blob/main/DFS_BFS_%EA%B8%B0%EC%B6%9C_%EB%AC%B8%EC%A0%9C%ED%92%80%EC%9D%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

특정 거리의 도시 찾기
----

풀이: N,M의 주어진 수가 매우 크기 때문에 완전 탐색은 별로인것 같다. 또한 *** 모든 간선의 길이가 1이라는 점** 에서 BFS 진행

In [ ]:
from collections import deque

n,m,k,x = map(int,input().split())

result =  [[0] for _ in range(n+1)]
for i in range(m):
  a,b = map(int,input().split())
  result[a].append(b)

distance = [-1]*(n+1) # 모든 최단 거리 테이블 초기화
distance[x] = 0 # 처음 출발하는 자기 자신은 0이라고 했으니

q = deque([x])

while q:
  now = q.popleft()
  # 현재 도시에서 이동 할 수 있는 모든 도시 확인
  for next in result[now]:
    if distance[next] == -1: # 아직 방문 되지 않은거라면
      distance[next] = distance[now]+1
      q.append(next)

check = False
for i in range(1,n+1):
  if distance[i] == k:
    print(i)
    check = True

if check == False:
  print("-1")







4 4 2 1
1 2
1 3
2 3
2 4
4


연구소 
---

풀이: 어디든 놓는걸 계산했을때 최대 64C3 인데 엄청 작은 수니깐 하나씩 DFS/BFS 확인해가면서 벽 세울 곳을 계산해도 괜찮다

이 문제는 완전탐색을 진행하면서 DFS/BFS 를 진행하는 문제라는데 두가지 방법 다 공부해보려고한다.

또한 이렇게 "바이러스가 사방으로 나간다" 처럼 주체가 방향성을 띄고있을때는 **동서남북 방향 벡터**를 사용해서 접근하는 것이 좋을 것 같다.

BFS 참고: https://hongcoding.tistory.com/m/130

In [ ]:
#BFS
from collections import deque
import copy

# 바이러스가 있는 자리 확인하는것 
def bfs():
  queue = deque()
  tmp = copy.deepcopy(graph)

  # 바이러스를 하나씩 확인하여 있는 자리 파악
  for i in range(n):
    for j in range(m):
      if tmp[i][j] == 2:
        queue.append((i,j))

  # 바이러스 확산 계산하는거 
  while queue:
    x,y =queue.popleft()
    for i in range(4):
      nx = x + dx[i]
      ny = y + dy[i]

      if nx<0 or nx>=n or ny <0 or ny >=m:
        continue
      if tmp[nx][ny] == 0:
        tmp[nx][ny] = 2
        queue.append((nx,ny))

    global answer 
    cnt = 0

    for i in range(n):
      cnt += tmp[i].count(0)
    
    answer = max(answer,cnt)

def makeWall(cnt):
    if cnt == 3:
        bfs()
        return

# 벽을 하나씩 다 넣어 보면서 확인하는 것 
    for i in range(n):
        for j in range(m):
            if graph[i][j] == 0:
                graph[i][j] = 1
                makeWall(cnt+1)
                graph[i][j] = 0

n, m = map(int, input().split())
graph = []
dx = [0, 0, 1, -1]
dy = [1, -1, 0, 0]

for i in range(n):
    graph.append(list(map(int, input().split())))

answer = 0
makeWall(0)
print(answer)

결국 두 방법다 마지막에 바이러스 접근 방식이 다른것 뿐이지 핵심은 두 방법다 조합을 하였을때 최대 발생 할 수 있는 연산 수가 작기때문에 일일히 다 한번씩 세워보고 계산하고 아니면 다시 다음걸로 넘어가고 하는 모든 완전 탐색이 가능하다는 점이였다.

In [ ]:
# DFS 사용
# DFS는 재귀적함수를 자주 이용한다.

n, m = map(int, input().split())
data = [] # 초기 맵

# 사방 이동 방향 벡터
dx = [0, 0, 1, -1]
dy = [1, -1, 0, 0]
temp = [[0]*m for _ in range(n)] # 벽 세운뒤 맵

for i in range(n):
    data.append(list(map(int, input().split())))


result = 0 

# DFS로 각 바이러스가 사방으로 퍼지게만들기
def virus(x,y):
  for i in range(4):
    nx = x+ dx[i]
    ny = y+ dy[i]

    # 상하좌우 중에서 바이러스가 퍼질수 있는 경우
    if nx >= 0 and nx<n and ny>= 0 and ny <m:
      if temp[nx][ny] == 0:
        # 해당 위치에 바이러스 배치, 재귀
        temp[nx][ny] == 2
        virus(nx,ny)

def safe():
  score  =0
  for i in range(n):
    for j in range(m):
      if temp[i][j] == 0:
        score += 1
  return score


def dfs(cnt):
  global result
  # 울타리가 3개 설치 된경우
  if cnt == 3:
    for i in range(n):
      for j in range(m):
        # 벽 세워진 그래프로 복사(위는 deepcopy를 사용)
        temp[i][j] =data[i][j]
    # 각자리에서 전파 시작
    for i in range(n):
      for j in range(m):
        if temp[i][j] == 2:
          virus(i,j)
    # 안전 구역 최대값 구함
    result = max(result,safe())
    return 

  for i in range(n):
    for j in range(m):
      if data[i][j] == 0:
        # 하나씩 설치해보고 풀었다가 하면서 최대값 찾기
        data[i][j] = 1
        cnt += 1
        dfs(cnt)
        data[i][j] = 0
        cnt -= 1
# 0개로 시작
dfs(0)

print(result)